In [6]:

'''
#Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
**Notes**
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

#max_features = 20000
max_features = 5000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80 #La longueur des séquences d'entrées
batch_size = 64 #inuitialement 32 
print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
# Taille des vectors 128
model.add(Embedding(max_features, 128))
#initialement dropout = 0.2, il permet de mettre randomly certains poids à zero
# 128 est le nombre d'unités memoire pour la couche LSTM
model.add(LSTM(100, dropout=0.5, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 84s 3ms/step - loss: 0.4828 - acc: 0.7689 - val_loss: 0.3721 - val_acc: 0.8366
Epoch 2/5
25000/25000 [==============================] - 81s 3ms/step - loss: 0.3681 - acc: 0.8419 - val_loss: 0.3602 - val_acc: 0.8413
Epoch 3/5
25000/25000 [==============================] - 81s 3ms/step - loss: 0.3254 - acc: 0.8620 - val_loss: 0.3992 - val_acc: 0.8317
Epoch 4/5
25000/25000 [==============================] - 81s 3ms/step - loss: 0.2990 - acc: 0.8741 - val_loss: 0.3730 - val_acc: 0.8365
Epoch 5/5
25000/25000 [==============================] - 14s 570us/step
Test score: 0.379233871383667
Test accuracy: 0.836400000038147


# Recurrent neural network : IMDB classification

On constate après une première exécution que le test loss est supérieur au training loss, Donc on va faire un overfit en augmentant le dropout et en diminuant la taille du réseaux.
#### Resultat 1 : loss: 0.0597  - val_loss: 0.7167 après 7 epochs

### Avec un dropout=0.5 et max_size = 5000 on a
#### Resultat 2 : loss: 0.2516 - val_loss: 0.3778 après 5 epochs

### Avec un dropout=0.5 et max_size = 5000 et batch_size = 64
#### Resultat 3 : loss: 0.2651 - val_loss: 0.3803

On obtient un accurracy de 83.6%.